In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip, Rotate # type: ignore


In [9]:
def load_data(path):
    train_x=sorted(glob(os.path.join(path,"training/images","*.tif")))
    train_y=sorted(glob(os.path.join(path,"training/1st_manual","*.gif")))
    
    test_x=sorted(glob(os.path.join(path,"test/images","*.tif")))
    test_y=sorted(glob(os.path.join(path,"test/1st_manual","*.gif")))

    return (train_x,train_y) , (test_x,test_y)
    

In [14]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [33]:
def augment_data(images,masks,save_path, augment=True):
    size=(512,512)


    for idx , (x,y) in tqdm(enumerate(zip(images,masks)), total=len(images)):

        # extracting name of data and mask

        xname=x.split("/")[-1].split(".")[0]
        

        # reading iamge and its mask:

        x = cv2.imread(x,cv2.IMREAD_COLOR)
        y = imageio.mimread(y)[0] 
        # print(x.shape, y.shape)


        if augment:
            
            aug=HorizontalFlip(p=1.0)
            augmented=aug(image=x,mask=y)
            x1=augmented["image"]
            y1=augmented["mask"]

            aug=VerticalFlip(p=1.0)
            augmented=aug(image=x,mask=y)
            x2=augmented["image"]
            y2=augmented["mask"]

            aug=Rotate(limit=45, p=1.0)
            augmented=aug(image=x,mask=y)
            x3=augmented["image"]
            y3=augmented["mask"]


            X = [x, x1,x2,x3]
            Y =[y,y1,y2,y3]




        else:

            X=[x]
            Y=[y]

        
        # save images:
        index=0
        for i , m in zip(X,Y):
            i=cv2.resize(i,size)
            m=cv2.resize(m,size)

            tmp_image_name=f"{xname}_{index}.png" 
            tmp_mask_name= f"{xname}_{index}.png" 

            image_path= os.path.join(save_path,"image",tmp_image_name)
            mask_path = os.path.join(save_path,"mask",tmp_mask_name)

            cv2.imwrite(image_path,i)
            cv2.imwrite(mask_path,m)

            index+=1

        



In [35]:
if __name__=="__main__":

    # seeding

    np.random.seed(42)
    data_path="RetinaData"
    (train_x,train_y) , (test_x,test_y)= load_data(data_path)

    print(f"Train: data:{len(train_x)} - mask:{len(train_y)}")
    print(f"Test: data:{len(test_x)} - mask:{len(test_y)}")


    # create Dirs for Augment
    create_dir("RetinaData/new_data/train/image")
    create_dir("RetinaData/new_data/train/mask")

    create_dir("RetinaData/new_data/test/image")
    create_dir("RetinaData/new_data/test/mask")


    #data Aug
    augment_data(train_x,train_y, "RetinaData/new_data/train",augment=True)
    augment_data(test_x,test_y, "RetinaData/new_data/test",augment=False)




     


Train: data:20 - mask:20
Test: data:20 - mask:20


100%|██████████| 20/20 [00:00<00:00, 88.74it/s]


In [36]:
import torch
import torch.nn as nn

In [75]:
class conv_block(nn.Module):

    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv1= nn.Conv2d(in_c,out_c, kernel_size=3, padding=1)
        self.bn1= nn.BatchNorm2d(out_c)

        self.conv2= nn.Conv2d(out_c,out_c, kernel_size=3, padding=1)
        self.bn2= nn.BatchNorm2d(out_c)

        self.relu=nn.ReLU()

    def forward(self,inputs):
        x=self.conv1(inputs)
        x=self.bn1(x)
        x=self.relu(x)

        x=self.conv2(x)
        x=self.bn2(x)
        x=self.relu(x)

        return x

class encoder_block(nn.Module):
    def __init__(self, in_c,out_c ):
        super().__init__()
        self.conv = conv_block(in_c,out_c)
        self.pool=nn.MaxPool2d((2,2))


    def forward(self, inputs):
        x=self.conv(inputs)
        p= self.pool(x)

        return x,p


class decoder_block(nn.Module):
    def __init__(self, in_c,out_c ):
        super().__init__()
        
        self.up = nn.ConvTranspose2d(in_c,out_c,kernel_size=2,stride=2, padding=0)
        self.conv=conv_block(out_c+out_c,out_c)


    def forward(self, inputs,skip):
        x= self.up(inputs)
        x=torch.cat([x,skip],axis=1) #concat on channels
        x=self.conv(x)

        return x


class build_unet(nn.Module):
    def __init__(self):
        super().__init__()

        # encoder
        self.e1=encoder_block(3,64)
        self.e2=encoder_block(64,128)
        self.e3=encoder_block(128,256)
        self.e4=encoder_block(256,512)

        #  Bottelneck

        self.b= conv_block(512,1024)

        #decoder
        self.d1=decoder_block(1024,512)
        self.d2=decoder_block(512,256)
        self.d3=decoder_block(256,128)
        self.d4=decoder_block(128,64)

        # classifier
        self.output= nn.Conv2d(64,1,kernel_size=1, padding=0)

    def forward(self,inputs):

        # encoder

        s1,p1=self.e1(inputs)
        s2,p2=self.e2(p1)
        s3,p3=self.e3(p2)
        s4,p4=self.e4(p3)


        #Bottelneck
        b=self.b(p4)

        #decoder

        d1=self.d1(b,s4)
        d2=self.d2(d1,s3)
        d3=self.d3(d2,s2)
        d4=self.d4(d3,s1)

        output=self.output(d4)

        return output






In [77]:
#test bulding model

x= torch.randn((2,3,512,512))
f=build_unet()

y=f(x)
print(y.shape)


torch.Size([2, 1, 512, 512])


In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

In [79]:
import os
import time
import random
import numpy as np
import cv2
import torch

""" Seeding the randomness. """
def seeding(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

""" Create a directory. """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Calculate the time taken """
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [82]:
from torch.utils.data import Dataset

class DriveDataset(Dataset):
    def __init__(self, images_path, masks_path):

        self.images_path = images_path
        self.masks_path = masks_path
        self.n_samples = len(images_path)

    def __getitem__(self, index):
        # Reading image
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = image/255.0 ## (512, 512, 3)
        image = np.transpose(image, (2, 0, 1))  ## (3, 512, 512)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)

        # Reading mask 
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
        mask = mask/255.0   ## (512, 512)
        mask = np.expand_dims(mask, axis=0) ## (1, 512, 512)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)

        return image, mask

    def __len__(self):
        return self.n_samples

In [83]:
def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0

    model.train()
    for x, y in loader:
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss = epoch_loss/len(loader)
    return epoch_loss

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

        epoch_loss = epoch_loss/len(loader)
    return epoch_loss

In [84]:
#training

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

seeding(42)

# create dir for checkpoints
create_dir("files")

#load dataset
train_x=sorted(glob("RetinaData/new_data/train/image/*"))
train_y=sorted(glob("RetinaData/new_data/train/mask/*"))

test_x=sorted(glob("RetinaData/new_data/test/image/*"))
test_y=sorted(glob("RetinaData/new_data/test/mask/*"))

# print(f"data size: train_x:{len(train_x)} - test_x:{len(test_x)}")


# Hyperparameters

H=512
W=512

size=(H,W)

batch_size=2
epochs=50
lr=1e-4
checkpoint_path="files/ch.pth"


# load dataset
""" Dataset and loader """
train_dataset = DriveDataset(train_x, train_y)
test_dataset = DriveDataset(test_x, test_y)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

device = torch.device('cuda')   ## RTX 3060
model = build_unet()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
loss_fn = DiceBCELoss()

""" Training the model """
best_valid_loss = float("inf")

for epoch in range(epochs):
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, loss_fn, device)
    valid_loss = evaluate(model, test_loader, loss_fn, device)

    """ Saving the model """
    if valid_loss < best_valid_loss:
        data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
        print(data_str)

        best_valid_loss = valid_loss
        torch.save(model.state_dict(), checkpoint_path)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
    data_str += f'\tTrain Loss: {train_loss:.3f}\n'
    data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
    print(data_str)

/home/aria/anaconda3/envs/torch/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Valid loss improved from inf to 1.3307. Saving checkpoint: files/ch.pth
Epoch: 01 | Epoch Time: 0m 16s
	Train Loss: 1.164
	 Val. Loss: 1.331

Valid loss improved from 1.3307 to 0.9809. Saving checkpoint: files/ch.pth
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.980
	 Val. Loss: 0.981

Valid loss improved from 0.9809 to 0.8900. Saving checkpoint: files/ch.pth
Epoch: 03 | Epoch Time: 0m 15s
	Train Loss: 0.921
	 Val. Loss: 0.890

Valid loss improved from 0.8900 to 0.8511. Saving checkpoint: files/ch.pth
Epoch: 04 | Epoch Time: 0m 16s
	Train Loss: 0.875
	 Val. Loss: 0.851

Valid loss improved from 0.8511 to 0.8118. Saving checkpoint: files/ch.pth
Epoch: 05 | Epoch Time: 0m 16s
	Train Loss: 0.835
	 Val. Loss: 0.812

Valid loss improved from 0.8118 to 0.7723. Saving checkpoint: files/ch.pth
Epoch: 06 | Epoch Time: 0m 15s
	Train Loss: 0.798
	 Val. Loss: 0.772

Valid loss improved from 0.7723 to 0.7495. Saving checkpoint: files/ch.pth
Epoch: 07 | Epoch Time: 0m 15s
	Train Loss: 0.761
	 Val. L

In [89]:
#testing model

from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from operator import add

def calculate_metrics(y_true, y_pred):
    #Ground truth
    y_true = y_true.cpu().numpy()
    y_true = y_true > 0.5
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    #Prediction 
    y_pred = y_pred.cpu().numpy()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    score_recall = recall_score(y_true, y_pred)
    score_precision = precision_score(y_true, y_pred)
    score_acc = accuracy_score(y_true, y_pred)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

def mask_parse(mask):
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)
    return mask


checkpoint_path = "files/ch.pth"
create_dir("results")
#Load the checkpoint
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = build_unet()
model = model.to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()

metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0]
time_taken = []

for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
    #Extract the name
    name = x.split("/")[-1].split(".")[0]

    #Reading image
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## (512, 512, 3)
    ## image = cv2.resize(image, size)
    x = np.transpose(image, (2, 0, 1))      ## (3, 512, 512)
    x = x/255.0
    x = np.expand_dims(x, axis=0)           ## (1, 3, 512, 512)
    x = x.astype(np.float32)
    x = torch.from_numpy(x)
    x = x.to(device)

    #Reading mask
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    ## mask = cv2.resize(mask, size)
    y = np.expand_dims(mask, axis=0)            ## (1, 512, 512)
    y = y/255.0
    y = np.expand_dims(y, axis=0)               ## (1, 1, 512, 512)
    y = y.astype(np.float32)
    y = torch.from_numpy(y)
    y = y.to(device)

    with torch.no_grad():
        #Prediction and Calculating FPS
        start_time = time.time()
        pred_y = model(x)
        pred_y = torch.sigmoid(pred_y)
        total_time = time.time() - start_time
        time_taken.append(total_time)


        score = calculate_metrics(y, pred_y)
        metrics_score = list(map(add, metrics_score, score))
        pred_y = pred_y[0].cpu().numpy()        ## (1, 512, 512)
        pred_y = np.squeeze(pred_y, axis=0)     ## (512, 512)
        pred_y = pred_y > 0.5
        pred_y = np.array(pred_y, dtype=np.uint8)

    #Saving masks
    ori_mask = mask_parse(mask)
    pred_y = mask_parse(pred_y)
    line = np.ones((size[1], 10, 3)) * 128

    cat_images = np.concatenate(
        [image, line, ori_mask, line, pred_y * 255], axis=1
    )
    cv2.imwrite(f"results/{name}.png", cat_images)

jaccard = metrics_score[0]/len(test_x)
f1 = metrics_score[1]/len(test_x)
recall = metrics_score[2]/len(test_x)
precision = metrics_score[3]/len(test_x)
acc = metrics_score[4]/len(test_x)
print(f"Jaccard: {jaccard:1.4f} - F1: {f1:1.4f} - Recall: {recall:1.4f} - Precision: {precision:1.4f} - Acc: {acc:1.4f}")

fps = 1/np.mean(time_taken)
print("FPS: ", fps)

/tmp/ipykernel_49773/1326212625.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_location=device))
100%|██████████|

Jaccard: 0.6601 - F1: 0.7949 - Recall: 0.7733 - Precision: 0.8233 - Acc: 0.9654
FPS:  775.5667939460618
